# Chat Engine - ReAct Mode

### Load data and build index

In [2]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, ServiceContext, LLMPredictor

In [17]:
data = SimpleDirectoryReader(input_dir="../data/paul_graham/").load_data()

In [18]:
index = GPTVectorStoreIndex.from_documents(data)

### Configure chat engine

In [30]:
chat_engine = index.as_chat_engine(chat_mode='react', verbose=True)

### Chat with your data

In [35]:
response = chat_engine.chat('Use tool and give me information about Paul Graham in a Markdown table')



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Query Engine Tool
Action Input: Information about Paul Graham in a Markdown table

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.



Observation: 
| Information | Description |
| ---------- | ----------- |
| Education | Attended RISD and dropped out in 1993 |
| Career | Worked at Interleaf, Viaweb, Y Combinator, and wrote Bel in Arc |
| Location | Moved to New York in 1993, then to England in 2016 |
| Art | Taught himself to paint and studied color theory at RISD |
| Writing | Wrote essays and kept a blog |
Thought: Do I need to use a tool? No
AI: Paul Graham is an influential figure in the tech and startup world. He is the co-founder of Y Combinator, a startup accelerator, and is known for his essays on technology and startups. He is also the author of the book Hackers & Painters.

> Finished chain.


In [37]:
print(response)

Paul Graham is an influential figure in the tech and startup world. He is the co-founder of Y Combinator, a startup accelerator, and is known for his essays on technology and startups. He is also the author of the book Hackers & Painters.


In [26]:
response = chat_engine.chat('What did I ask you?')



> Entering new AgentExecutor chain...
AI: You asked me who Paul Graham is.

> Finished chain.


In [27]:
print(response)

You asked me who Paul Graham is.
